In [3]:
import findspark
findspark.init('/home/ubuntu/spark-2.3.2-bin-hadoop2.7')
import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp_project').getOrCreate()

In [6]:
data = spark.read.csv(
    '../Python-and-Spark-for-Big-Data/Spark_for_Machine_Learning/Natural_Language_Processing/smsspamcollection/SMSSpamCollection', 
    inferSchema=True, sep='\t')

In [7]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [8]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [9]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [10]:
from pyspark.sql.functions import length

In [11]:
data = data.withColumn('length', length(data['text']))

In [12]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [13]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [14]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [16]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='vec_token')
idf = IDF(inputCol='vec_token', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [17]:
from pyspark.ml.feature import VectorAssembler

In [19]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [20]:
from pyspark.ml.classification import NaiveBayes

In [21]:
nb = NaiveBayes()

In [23]:
from pyspark.ml import Pipeline

In [28]:
prep_pipe = Pipeline(stages=[ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up])

In [29]:
cleaned_data = prep_pipe.fit(data).transform(data)

In [30]:
cleaned_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'vec_token',
 'tf_idf',
 'features']

In [31]:
cleaned_data = cleaned_data.select('label', 'features')

In [32]:
cleaned_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,297,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,139...|
|  0.0|(13424,[10,53,103...|
|  0.0|(13424,[125,184,4...|
|  1.0|(13424,[1,47,118,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,120...|
|  1.0|(13424,[8,17,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,96,217...|
|  0.0|(13424,[552,1697,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,47...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [33]:
train, test = cleaned_data.randomSplit([.7, .3])

In [34]:
spam_detector = nb.fit(train).transform(test)

In [35]:
spam_detector.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,5,20,...|[-799.52158511222...|[1.0,6.5217157568...|       0.0|
|  0.0|(13424,[0,1,11,32...|[-871.83419111400...|[1.0,1.4074297000...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-216.33406493382...|[1.0,1.7973081193...|       0.0|
|  0.0|(13424,[0,1,15,20...|[-672.67441592821...|[1.0,2.3481655157...|       0.0|
|  0.0|(13424,[0,1,27,35...|[-1487.4325585233...|[0.99999996271718...|       0.0|
|  0.0|(13424,[0,1,27,88...|[-1527.8950524421...|[1.13054782917578...|       1.0|
|  0.0|(13424,[0,1,43,69...|[-614.13989200886...|[0.99951740677658...|       0.0|
|  0.0|(13424,[0,1,72,10...|[-699.19220091985...|[1.0,1.8892653404...|       0.0|
|  0.0|(13424,[0,1,146,1...|[-253.98911858044...|[0.20336453565756...|       1.0|
|  0.0|(13424,[0

In [36]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [37]:
acc_eval = MulticlassClassificationEvaluator()

In [39]:
acc = acc_eval.evaluate(spam_detector)

In [40]:
print('ACC of NB Model:')
print(acc)

ACC of NB Model:
0.922644183087506
